<a href="https://colab.research.google.com/github/bruceMacLeod/COS475-575/blob/main/Lab/CVDPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> A Study of Factors related to Cardiovascular risk in Adults </center>


The objective of this notebook is to show some of the essential steps of a workflow for building predictive models. The notebook provides a few examples of each step and it is only a very thin slice of what a complete analysis would consist of. 

The workflow includes:
1. **Problem Definition**:  A clear definition of the problem enables us to identify the appropriate data to gather and technique(s) to use in order to solve the problem. For many problems this many require background reading, discussion with domain experts, and layered problem specification. 
2. **Data Gathering**: We have to know which data to use, where to gather them, and how to make them useful to solve our problem. In many cases, data from multiple sources can provide deeper insights. 
3. **Data Cleaning and Wrangling**: Raw data are generally incomplete, inconsistent, and contain many errors. Thus, we need to prepare the data for further processing. Data wrangling is the process of cleaning, structuring, and enriching raw data into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes, such as analytics.
4. **Exploratory Data Analysis**: Exploratory data analysis (EDA) is an approach of performing initial investigations on our data. EDA normally has descriptive nature and uses graphical statistics to discover patterns, to identify anomalies, to test hypothesis, and to check assumptions regarding our data. 

5. **Data Modelling**:  Data modelling involves selecting and optiming the machine learning models that generate the best predictive performance based on the data we have. 
6. **Prediction**: Once we have developed the best predictive model, we can deploy it to make predictions.


References : 
1. https://github.com/richasethi3/CVD_Prediction
2. https://www.kaggle.com/ar2017/titanic-end-to-end-ml-workflow-top-7/notebook
3. https://github.com/bruceMacLeod/COS475-575/blob/main/Assignment/HypertensionV1.ipynb

# Problem Definition

Cardiovascular disease (CVD) is a general term for conditions affecting the heart or blood vessels. It's usually associated with a build-up of fatty deposits inside the arteries (atherosclerosis) and an increased risk of blood clots. It can also be associated with damage to arteries in organs such as the brain, heart, kidneys and eyes.

CVD is one of the main causes of death and disability in the USA, but it can often largely be prevented by leading a healthy lifestyle.

The exact cause of CVD isn't clear, but there are lots of things that can increase your risk of getting it. These are called "risk factors". The more risk factors you have, the greater your chances of developing CVD. 

1.  **High blood pressure** (hypertension) is one of the most important risk factors for CVD. If your blood pressure is too high, it can damage your blood vessels.

2.   **Smoking** Smoking and other tobacco use is also a significant risk factor for CVD. The harmful substances in tobacco can damage and narrow your blood vessels.

3.   **High cholesterol** Cholesterol is a fatty substance found in the blood. If you have high cholesterol, it can cause your blood vessels to narrow and increase your risk of developing a blood clot.


4.   **Diabetes** Diabetes is a lifelong condition that causes your blood sugar level to become too high. High blood sugar levels can damage the blood vessels, making them more likely to become narrowed.

5.    **Inactivity** If you don't exercise regularly, it's more likely that you'll have high blood pressure, high cholesterol levels and be overweight. All of these are risk factors for CVD.

6.    **Being overweight or obese** Being overweight or obese increases your risk of developing diabetes and high blood pressure, both of which are risk factors for CVD.

We apply the tools of machine learning to predict the factors that are associated with cardiovascular disease in adults.



# Data Gathering and Import


In [1]:
#Before moving to the next section, we need to import all packages required to do the analysis by calling the following:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import tarfile
import urllib
import logging

from functools import reduce

from sklearn.impute import SimpleImputer 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix



### Gathering and Importing Data

We need some functions to help automate the large number of files 

In [ ]:
def download_data(data_dir, file_list):

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    for (year, data_file) in file_list:
        sub_dir = os.path.join(data_dir, year)
        if not os.path.exists(sub_dir):
            os.makedirs(sub_dir)
        url = 'http://wwwn.cdc.gov/Nchs/Nhanes/{0}/{1}.XPT'.format(year, data_file)
        file_name = os.path.join(sub_dir, data_file + '.XPT')
        if not os.path.exists(file_name):
            logging.info('Downloading: {}'.format(url))
            urllib.request.urlretrieve(url, file_name)
        else:
            logging.info('File exists: {}'.format(file_name))
            
def read_data_from_row(offset,ncols,col_list):
    df = pd.DataFrame()
    for i in range(ncols): 
        filename = LOCAL_DATA_PATH + file_list[offset + i][0] + "/" + file_list[offset + i][1] + ".XPT"
        one_year_df = pd.read_sas(filename)
        df = pd.concat([df,one_year_df], axis=0)
    df = df.loc[:, col_list]
    return df

In [ ]:
file_list = [
        ('2015-2016', 'DEMO_I'),    ('2017-2018', 'DEMO_J'),  ('2013-2014', 'DEMO_H'),
        ('2015-2016', 'BPX_I'),     ('2017-2018', 'BPX_J'),   ('2013-2014', 'BPX_H'),
        ('2015-2016', 'BMX_I'),     ('2017-2018', 'BMX_J') ,  ('2013-2014', 'BMX_H'),
        ('2015-2016', 'TCHOL_I'),   ('2017-2018', 'TCHOL_J'), ('2013-2014', 'TCHOL_H'),
        ('2015-2016', 'DIQ_I'),     ('2017-2018', 'DIQ_J'),   ('2013-2014', 'DIQ_H'),
        ('2015-2016', 'SMQ_I'),     ('2017-2018', 'SMQ_J'),   ('2013-2014', 'SMQ_H'),
        ('2015-2016', 'MCQ_I'),     ('2017-2018', 'MCQ_J'),   ('2013-2014', 'MCQ_H'),
        ('2015-2016', 'HDL_I'),     ('2017-2018', 'HDL_J'),   ('2013-2014', 'HDL_H'),
        ('2015-2016', 'TRIGLY_I'),  ('2017-2018', 'TRIGLY_J'),('2013-2014', 'TRIGLY_H'),
        ('2015-2016', 'KIQ_U_I'),   ('2017-2018', 'KIQ_U_J'), ('2013-2014', 'KIQ_U_H')
    ]

demo_cols = ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'INDFMIN2']
bpx_cols = ['SEQN', 'BPXPULS','BPXSY1', 'BPXDI1']
bmx_cols = ['SEQN', 'BMXBMI', 'BMXWAIST']
tchol_cols = ['SEQN', 'LBXTC']
diab_cols = ['SEQN', 'DIQ010']
smoking_cols = ['SEQN', 'SMQ020']
heart_cols = ['SEQN', 'MCQ160B', 'MCQ160C', 'MCQ160D', 'MCQ160E', 'MCQ160F', 'MCQ300A']
hdl_cols = ['SEQN', 'LBDHDD']
trigly_cols = ['SEQN', 'LBXTR', 'LBDLDL']
kidney_cols = ['SEQN', 'KIQ022']


In [ ]:
LOCAL_DATA_PATH = os.path.join("datasets", "nhanes") + "/"

download_data(LOCAL_DATA_PATH, file_list)

In [ ]:
demo_df = read_data_from_row(0,3,demo_cols)
bpx_df = read_data_from_row(3,3,bpx_cols)
bmx_df = read_data_from_row(6,3,bmx_cols)
tchol_df = read_data_from_row(9,3,tchol_cols)
diab_df = read_data_from_row(12,3,diab_cols)
smoking_df = read_data_from_row(15,3,smoking_cols)
heart_df = read_data_from_row(18,3,heart_cols)
hdl_df = read_data_from_row(21,3,hdl_cols)
trigly_df = read_data_from_row(24,3,trigly_cols)
kidney_df = read_data_from_row(27,3,kidney_cols)

### Merge the datatables into a single table

In [ ]:
pdList = [demo_df, bpx_df, bmx_df, hdl_df, trigly_df, tchol_df, diab_df,
          kidney_df, heart_df, smoking_df]
cvd_df = reduce(lambda x,y: pd.merge(x,y, on='SEQN', how='outer'), pdList)

In [ ]:
#rename the columns to make the headers to something more meaningful
cvd_df.rename(columns={'SEQN': 'seqn', 'RIAGENDR': 'gender', 'RIDAGEYR':'age',
                   'RIDRETH1':'ethnicity', 'INDFMIN2':'income', 'BPXPULS':'pulse_regular',
                   'BPXSY1':'sysbp', 'BPXDI1':'diabp', 'BMXBMI':'bmi',
                   'BMXWAIST':'waistcircum', 'LBDHDD':'hdl', 'LBXTR':'trigly',
                   'LBDLDL':'ldl', 'LBXTC':'totchol', 'DIQ010':'diabetes',
                   'KIQ022':'kidney_fail', 'MCQ160B':'congestive_fail', 'MCQ160C':'coronary_disease',
                   'MCQ160D':'angina', 'MCQ160E':'heart_attack',
                   'MCQ160F':'stroke', 'MCQ300A':'fam_history', 'SMQ020':'smoking'}, inplace=True)
cvd_df.set_index('seqn', inplace=True)
cvd_df.head()

###  Exploring Data Structure and Features
Before performing data analysis, we often need to know the structure of our data. Therefore, we perform the following:
- Viewing a small part of our datasets
- Viewing data shape
- Describing the features contained in the datasets

In [ ]:
cvd_df.info()

In [ ]:
# Only want to analysis on adults

cvd_df.info()

In [ ]:
#count and find the percentage of null values and concatenat the results
missing = pd.concat([cvd_df.isnull().sum(), 100*cvd_df.isnull().mean()], axis=1)
missing.columns = ['count', 'percentage']
missing.sort_values(by='count', ascending=False)

# Data Cleaning and Wrangling

## Starting with the missing values

In [ ]:
cvd_df.shape

In [ ]:
#filering out the ldl null values from the dataset
cvd_df = cvd_df[cvd_df['ldl'].notna()].reset_index(drop=True)

In [ ]:
cvd_df.shape

In [ ]:
#look at the count and percentage of missing values again
missing = pd.concat([cvd_df.isnull().sum(), 100*cvd_df.isnull().mean()], axis=1)
missing.columns = ['count', 'percentage']
missing.sort_values(by='count', ascending=False)

Taking out the missing ldl values has taken care of missing trigly values and the rest of the missing columns as well.

In [ ]:
cvd_df = cvd_df[cvd_df['heart_attack'].notna()].reset_index(drop=True)
#look at the count and percentage of missing values again
missing = pd.concat([cvd_df.isnull().sum(), 100*cvd_df.isnull().mean()], axis=1)
missing.columns = ['count', 'percentage']
missing.sort_values(by='count', ascending=False)

This has resulted in reduction of our dataset by over 75%. Let's impute the rest of the missing sysbp, diabp, bmi and waistcircum columns by the median. Impute pulse_regular by mode, and forward fill the income column.

In [ ]:
cvd_df.shape

In [ ]:
cvd_df[['sysbp', 'diabp', 'bmi', 'waistcircum']] = cvd_df[['sysbp', 'diabp', 'bmi',
                                                           'waistcircum']].fillna(cvd_df[['sysbp', 'diabp',
                                                                                          'bmi', 'waistcircum']].median())

In [ ]:
cvd_df['pulse_regular'].fillna(cvd_df['pulse_regular'].mode()[0], inplace=True)

In [ ]:
cvd_df['income'].fillna(method='ffill', inplace=True)

In [ ]:
cvd_df.dtypes

In [ ]:
dtype_cols = [col for col in cvd_df.columns if col not in ['bmi', 'waistcircum']]

for col in dtype_cols:
    cvd_df[col] = cvd_df[col].astype('int')
cvd_df.dtypes

In [ ]:
#let's get the histograms to get an idea of their distributions
cvd_df.hist(figsize=(15, 10), color='rebeccapurple')
plt.subplots_adjust(hspace=0.5)
plt.show()

### filter out rows, columns, recode 

In [ ]:
def hyper(sbp, dbp):
    if ((sbp <= 130) and (dbp <= 80)):    
        return 0
    else:
        return 1

In [ ]:
#def a function which takes different heart condions as input and returns 1 if either one is true
#this is going to be our target
def CVD(heart_1, heart_2, heart_3, heart_4, heart_5):
    if ((heart_1 == 1) or (heart_2 == 1) or (heart_3 == 1)
       or (heart_4 == 1) or (heart_5 == 1)):
        return 1
    else:
        return 0

In [ ]:
def cvd_recode_values(cvd_df):
    #replace the 1(male) from the original dataset to 0 and 2(female) to 1
    cvd_df['gender'].replace({1: 0, 2: 1}, inplace=True)
 
    # replace 13(under $20,000) by 4($15,000 to $19,999)
    #replace 12(over $20,000), 77(refused) and 99(don't know) by mode($25,000 to $34,999). 
    cvd_df['income'].replace({13: 4, 12: 6, 77: 6, 99: 6}, inplace=True)
 
    #Here 1 means the individual has been told they have diabetes, 2 means no diabetes, 3 means borderline and 9 stands for refused
    #replace 3 by 1 and 9 by 2
    cvd_df['diabetes'].replace({3: 1, 9: 2}, inplace=True)

    #Here 1 means the individual has had kidney failure, 2 means no kidney failure and 9 stands for refused
    #replace 9 by 2
    cvd_df['kidney_fail'].replace({9: 2}, inplace=True)

    # Here 1 means the individual has had congestive heart failure, 2 means no congestive heart failure and 9 stands for refused.
    #replace 9 by 2
    cvd_df['congestive_fail'].replace({9: 2}, inplace=True)

    #Here 1 means the individual has had coronary heart disease, 2 means no congestive coronary heart disease and 9 stands for refused.
    cvd_df['coronary_disease'].replace({9: 2, 7: 2}, inplace=True)

    #Here 1 means the individual has had angina, 2 means no angina and 9 stands for refused.
    cvd_df['angina'].replace({9: 2}, inplace=True)

    # Here 1 means the individual has had heart attack, 2 means no heart attack and 9 stands for refused.
    #replace 9 by 2
    cvd_df['heart_attack'].replace({9: 2}, inplace=True)

    #Here 1 means the individual has had stroke, 2 means no stroke and 9 stands for refused.
    #replace 9 by 2
    cvd_df['stroke'].replace({9: 2}, inplace=True)

    #Here 1 means the individual has family history of heart disease, 2 means no family history of heart disease, 7 means don't know and 9 stands for refused.
    #replace 9 and 7 by 2
    cvd_df['fam_history'].replace({9: 2, 7: 2}, inplace=True)

    #Here 1 means the individual smokes, 2 means no smoking, 7 means don't know and 9 stands for refused.
    #replace 9 and 7 by 2
    cvd_df['smoking'].replace({9: 2, 7: 2}, inplace=True)
    return cvd_df


In [ ]:
def cvd_add_attributes(cvd_df):
    if ('congestive_fail' in cvd_df.columns) &  ('coronary_disease' in cvd_df.columns) & \
       ('angina' in cvd_df.columns) & ('heart_attack' in cvd_df.columns) & \
       ('stroke' in cvd_df.columns) :
       cvd_df['CVD_risk'] = cvd_df.apply(lambda x: CVD(x['congestive_fail'], x['coronary_disease'],
                                         x['angina'], x['heart_attack'], 
                                         x['stroke']), axis=1)
    if ('sysbp' in cvd_df.columns) & ('diabp' in cvd_df.columns) : 
       cvd_df['hypertension_cat'] = cvd_df.apply(lambda x : hyper(x['sysbp'], x['diabp']), axis=1)
    return cvd_df

In [ ]:
def cvd_trim_rows(cvd_df):
    # only want to do the analysis on adults
    if ('age' in cvd_df.columns):
        cvd_df = cvd_df[cvd_df.age >=20]
    return cvd_df    

In [ ]:
def cvd_drop_columns(cvd_df):
        if ('congestive_fail' in cvd_df.columns) &  ('coronary_disease' in cvd_df.columns) & \
           ('angina' in cvd_df.columns) & ('heart_attack' in cvd_df.columns) & \
           ('stroke' in cvd_df.columns) :
            cvd_df = cvd_df.drop(columns=['congestive_fail', 'coronary_disease', 'angina', 'heart_attack', 'stroke'], axis=1)# We do not need the seqn now (only needed for the merge)
        if ('sysbp' in cvd_df.columns) & ('diabp' in cvd_df.columns) : 
            cvd_df = cvd_df.drop(columns=['sysbp','diabp'],axis=1)
        return cvd_df


In [ ]:
def cvd_add_trim__recode_drop(cvd_df):
    cvd_df = cvd_trim_rows(cvd_df)
    cvd_df = cvd_add_attributes(cvd_df)
    cvd_df = cvd_recode_values(cvd_df)
    cvd_df = cvd_drop_columns(cvd_df)
    
    return cvd_df

In [ ]:
cvd_df.head()

In [ ]:
cvd_df = cvd_add_trim__recode_drop(cvd_df)
cvd_df.head()

In [ ]:
#get the new histograms to get an idea of their distributions
cvd_df.hist(figsize=(20, 15), color='rebeccapurple')
plt.subplots_adjust(hspace=0.5)
plt.xticks(fontsize=12, color='black')
plt.yticks(fontsize=12, color='black')
plt.show()

In [ ]:
#setting up a datapath for our file
if not os.path.exists(LOCAL_DATA_PATH):
    os.mkdir(datapath)
datapath_cvd_data = os.path.join(LOCAL_DATA_PATH, 'cvd_data_cleaned.csv')
if not os.path.exists(datapath_cvd_data):
    cvd_df.to_csv(datapath_cvd_data, index=False)

### Now split into training and test data sets

In [ ]:
y = cvd_df['CVD_risk']
X = cvd_df.drop('CVD_risk',axis=1)

In [ ]:
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train,  X_val, y_train, y_val = train_test_split(X_train,y_train,  test_size=0.1, random_state=42)

In [ ]:
#confirming the ratios of train, test and validation sets for X
print('Percent heldout for training:', round(100*(len(X_train)/len(cvd_df)),0),'%')
print('Percent heldout for validation', round(100*(len(X_val)/len(cvd_df)),0),'%')
print('Percent heldout for testing:', round(100*(len(X_test)/len(cvd_df)),0),'%')

In [ ]:
#confirming the ratios of train, test and validation sets for y
print('Percent heldout for training:', round(100*(len(y_train)/len(cvd_df)),0),'%')
print('Percent heldout for validation:', round(100*(len(y_val)/len(cvd_df)),0),'%')
print('Percent heldout for testing:', round(100*(len(y_test)/len(cvd_df)),0),'%')

# Exploratory Data Analysis 

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
y_val.value_counts(normalize=True)

Our data is highly imbalanced with about 11% individuals with CVD risk. We may have to take this into account while modeling

In [ ]:

Xy_train = X_train.copy()
Xy_train['CVD_risk'] = y_train
Xy_grouped = Xy_train.groupby('CVD_risk')[['age',  'bmi', 'waistcircum',
                                   'hdl', 'trigly', 'ldl', 'totchol']].mean().reset_index()
Xy_grouped

In [ ]:
#def a function to plot barplots for the mean of grouped features
def barplots(columns, ncol=None, figsize=(15, 8)):
    if ncol is None:
        ncol = len(columns)
    nrow = int(np.ceil(len(columns) / ncol))
    fig, axes = plt.subplots(nrow, ncol, figsize=figsize, squeeze=False)
    fig.subplots_adjust(wspace=0.3, hspace=0.6)
    for i, col in enumerate(columns):
        ax = axes.flatten()[i]
        barlist=ax.bar(x = 'CVD_risk', height = col, data=Xy_grouped)
        barlist[0].set_color('forestgreen')
        barlist[1].set_color('darkred')
        ax.set_xticks([0, 1])
        ax.set_xticklabels(['No', 'Yes'], fontsize=14, color='black')
        ax.set_xlabel('CVD risk',fontsize=14, color='black')
        ax.set_ylabel(col, fontsize=14, color='black')
    nsubplots = nrow * ncol    
    for empty in range(i+1, nsubplots):
        axes.flatten()[empty].set_visible(False)

In [ ]:
features = [i for i in Xy_grouped.columns if i not in ['CVD_risk']]

In [ ]:
barplots(features, ncol=4, figsize=(15, 12))

In [ ]:
Xy_train.columns

In [ ]:
# Adding heatmaps
corr = Xy_train[['gender', 'age',  'bmi', 'waistcircum', 'hdl', 'ldl', 'trigly', 'totchol',
               'diabetes', 'kidney_fail', 'fam_history', 'smoking', 'CVD_risk']].corr()
plt.figure(figsize=(24,10))
heatmap = sns.heatmap(corr, vmin=-1, vmax=1, cmap='BrBG', annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':20}, pad=12)
plt.xticks(fontsize=14, color='black')
plt.yticks(fontsize=14, color='black')

### More graph and EDA needed (todo)

## Setup the pipeline to normalize values

### set missing values of numerical data to the median

In [ ]:
X_train.head()

In [ ]:
X_train.columns

In [ ]:
num_attribs = ['age', 'income', 'bmi', 'waistcircum', 'hdl', 'trigly', 'ldl', 'totchol']
cat_attribs = ["gender","ethnicity", "pulse_regular", 'diabetes', 'kidney_fail', 'fam_history', 'smoking', 'hypertension_cat']

#cvd_num = cvd_df.drop(cat_attribs,axis=1)

In [ ]:

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
        ])

In [ ]:

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
        ])
cvd_prepared = full_pipeline.fit_transform(X_train)

In [ ]:
cvd_prepared.shape

In [ ]:
cvd_prepared[1,0:27]

### Create a dataframe from the prepared numpy array so we can view data

In [ ]:
cvd_df[cat_attribs].apply(pd.Series.value_counts)

In [ ]:
column_names = num_attribs.copy()
column_names.extend(['Male','Female',
                     'Mexican','Hispanic','White','Black','Other',
                     'Pulse_regular', 'Pulse_irreg',
                     'diabetes','no_diabetes',
                     'kidney_fail', 'no_kidney_fail',
                     'fam_history', 'no_fam_hist',
                     'smoking', 'no_smoking',
                     'hypertension', 'no_hypertension'])
cvd_prepared_df = pd.DataFrame(cvd_prepared, columns=column_names)
cvd_prepared_df.T

#  Data Modeling

### Stochastic Gradient Descent

In [ ]:

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
sgd_clf.fit(cvd_prepared, y_train)

In [ ]:
some_data = cvd_prepared[5]
some_labels = y_train.iloc[5]
sgd_clf.predict([some_data])

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, cvd_prepared, y_train, cv=3, scoring="accuracy")

## Confusion Matrix

### Exercise : Construct a confusion matrix

### Exercise : Provide your thoughts on the numbers in the confusion matrix, including an analysis of why things may not be working out so well. 

## Precision and Recall

### Exercise : Calculate the Precision, Recall, and F1-score

### Provide your thoughts on precision, recall, and F1-score, including an analysis of why things may not be working out so well. 

## Precision/Recall Trade-off

### Exercise : Construct a graph of the Precision and Recall values as the threshold changes  

### Exercise : Why does the precision curve look jagged ?

## The ROC Curve

### Exercise : Construct and ROC curve

### Graduate Students/Extra Credit : Choose another modeling technique, compare results on the ROC curve 